In [ ]:
# London Bike Riders Data Analysis (2015–2016)
# Notebook: /notebooks/01_etl_cleaning.ipynb
# Goal: Load -> Clean -> Feature engineer -> Export for Power BI (model + optional star schema)

import os
import numpy as np
import pandas as pd

# -----------------------------
# 0) Paths (edit)
# -----------------------------
RAW_PATH = "london_bike_sharing_dataset.csv"
OUT_DIR = "../data/processed"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------------
# 1) Load
# -----------------------------
df = pd.read_csv(RAW_PATH)
print("Raw shape:", df.shape)
df.head()

# -----------------------------
# 2) Standardize columns
# -----------------------------
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

# Rename common variants to consistent names (edit mapping to match your file)
rename_map = {
    "date": "dt",
    "datetime": "dt",
    "timestamp": "dt",
    "cnt": "ride_count",
    "count": "ride_count",
    "riders": "ride_count",
    "temp": "temp_c",
    "temperature": "temp_c",
    "hum": "humidity",
    "humidity": "humidity",
    "windspeed": "wind_speed",
    "wind_speed": "wind_speed",
    "season": "season",
    "weather": "weather",
    "weather_code": "weather",
    "is_holiday": "is_holiday",
    "holiday": "is_holiday",
    "is_weekend": "is_weekend",
    "weekend": "is_weekend",
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

print("Columns:", df.columns.tolist())

# Required columns
required = ["dt", "ride_count"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# -----------------------------
# 3) Types
# -----------------------------
df["dt"] = pd.to_datetime(df["dt"], errors="coerce")
df["ride_count"] = pd.to_numeric(df["ride_count"], errors="coerce")

for col in ["temp_c", "humidity", "wind_speed"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

for col in ["season", "weather"]:
    if col in df.columns:
        df[col] = df[col].astype("string").str.strip()

for col in ["is_holiday", "is_weekend"]:
    if col in df.columns:
        # Accept 0/1, True/False, "Yes/No"
        df[col] = (
            df[col]
            .replace({"Yes": 1, "No": 0, "TRUE": 1, "FALSE": 0, True: 1, False: 0})
        )
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

# -----------------------------
# 4) Basic cleaning
# -----------------------------
before = len(df)
df = df[df["dt"].notna()]
df = df[df["ride_count"].notna() & (df["ride_count"] >= 0)]
after = len(df)
print("Dropped:", before - after)

# Optional: filter target range (adjust if your dataset differs)
df = df[(df["dt"] >= "2015-01-01") & (df["dt"] <= "2016-12-31")]

# -----------------------------
# 5) Feature engineering for analysis + Power BI
# -----------------------------
df["date_key"] = df["dt"].dt.date
df["year"] = df["dt"].dt.year
df["month"] = df["dt"].dt.month
df["month_name"] = df["dt"].dt.month_name()
df["quarter"] = df["dt"].dt.quarter
df["weekday_name"] = df["dt"].dt.day_name()
df["weekday_num"] = df["dt"].dt.weekday + 1  # Monday=1
df["hour"] = df["dt"].dt.hour

# If is_weekend is missing, derive it
if "is_weekend" not in df.columns:
    df["is_weekend"] = (df["dt"].dt.weekday >= 5).astype(int)

df["day_type"] = np.where(df["is_weekend"] == 1, "Weekend", "Weekday")

# Normalize humidity to percent if 0..1
if "humidity" in df.columns:
    df["humidity_pct"] = np.where(
        df["humidity"].isna(),
        np.nan,
        np.where((df["humidity"] >= 0) & (df["humidity"] <= 1), df["humidity"] * 100, df["humidity"])
    )
else:
    df["humidity_pct"] = np.nan

# Weather/season defaults
if "season" not in df.columns:
    df["season"] = "Unknown"
if "weather" not in df.columns:
    df["weather"] = "Unknown"

# -----------------------------
# 6) Quick QA (use numbers in README insights)
# -----------------------------
print("Rows:", len(df))
print("Date range:", df["dt"].min(), "->", df["dt"].max())
print("Avg rides:", df["ride_count"].mean())
print("Weekday vs Weekend:")
print(df.groupby("day_type")["ride_count"].agg(["count", "mean", "sum"]).sort_values("sum", ascending=False))

if "temp_c" in df.columns:
    print("Rides by temperature quartile:")
    df["temp_band"] = pd.qcut(df["temp_c"], 4, duplicates="drop")
    print(df.groupby("temp_band")["ride_count"].mean())

# -----------------------------
# 7) Export (single-table model for Power BI)
# -----------------------------
model_cols = [
    "date_key","year","month","month_name","quarter","weekday_name","day_type","hour",
    "is_holiday","is_weekend","season","weather",
    "ride_count","temp_c","humidity_pct","wind_speed"
]
model_cols = [c for c in model_cols if c in df.columns]
df_model = df[model_cols].copy()

out_model = os.path.join(OUT_DIR, "london_bike_model_2015_2016.csv")
df_model.to_csv(out_model, index=False)
print("Saved:", out_model, "shape:", df_model.shape)

# -----------------------------
# 8) Optional: Star schema exports (recommended)
# -----------------------------
dim_date = (
    df_model[["date_key","year","month","month_name","quarter","weekday_name","day_type"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_weather = (
    df_model[["season","weather"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_weather["weather_key"] = dim_weather.index + 1

fact = df_model.merge(dim_weather, on=["season","weather"], how="left")

fact_cols = [
    "date_key","weather_key","hour","is_holiday",
    "ride_count","temp_c","humidity_pct","wind_speed"
]
fact_cols = [c for c in fact_cols if c in fact.columns]
fact_rides = fact[fact_cols].copy()

dim_date.to_csv(os.path.join(OUT_DIR, "dim_date.csv"), index=False)
dim_weather.to_csv(os.path.join(OUT_DIR, "dim_weather.csv"), index=False)
fact_rides.to_csv(os.path.join(OUT_DIR, "fact_rides.csv"), index=False)

print("Saved star schema:",
      os.path.join(OUT_DIR, "dim_date.csv"),
      os.path.join(OUT_DIR, "dim_weather.csv"),
      os.path.join(OUT_DIR, "fact_rides.csv"))
